In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))

import pandas as pd
from pycaret.classification import ClassificationExperiment
import src.utils as utils
import src.calcEMA as calc_utils
import src.myenv as myenv
from datetime import datetime

from itertools import combinations
import plotly.express as px

<built-in method __dir__ of sqlite3.Connection object at 0x7f9a493b0740>


In [2]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'BTCUSDT'
interval = '1m'
# lightgbm  xgboost lr et rf
estimator = 'knn'
_compare_models = False

start_train_date = '2022-06-01'  # train < and test >=
start_test_date = '2023-01-01'  # train < and test >=

stop_loss = 1.0
# regression_times = 0  # 24 * 30 * 2  # horas
times_regression_PnL = 120
normalize = True
use_gpu = False
tune_model = False
apply_combination_features = True

### Metadata

<code>
Field Name - Description</br>
open_time - Kline Open time in unix time format</br>
open - Open Price</br>
high - High Price</br>
low	- Low Price</br>
close	- Close Price</br>
volume - Volume</br>
close_time - Kline Close time in unix time format</br>
quote_volume - Quote Asset Volume</br>
count	- Number of Trades</br>
taker_buy_volume - Taker buy base asset volume during this period</br>
taker_buy_quote_volume - Taker buy quote asset volume during this period</br>
ignore - Ignore</br>
</code>

In [3]:
cols = myenv.all_klines_cols.copy()
cols.remove('ignore')
data = utils.get_data(symbol=symbol, save_database=False, interval=interval, tail=-1, columns=cols, parse_dates=True, start_date=start_train_date)
data = data[data['open_time'] >= start_train_date]
data = utils.parse_type_fields(data, parse_dates=True)
data = utils.adjust_index(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 332550 entries, 2022-06-01 00:00:08.192000 to 2023-10-16 22:54:00
Data columns (total 12 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   open_time                     332550 non-null  datetime64[ns]
 1   open                          332550 non-null  float32       
 2   high                          332550 non-null  float32       
 3   low                           332550 non-null  float32       
 4   close                         332550 non-null  float32       
 5   volume                        332550 non-null  float32       
 6   close_time                    332550 non-null  int64         
 7   quote_asset_volume            332550 non-null  float32       
 8   number_of_trades              332550 non-null  int16         
 9   taker_buy_base_asset_volume   332550 non-null  float32       
 10  taker_buy_quote_asset_volume  332550 no

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol
ix_open_time,,,,,,,,,,,,
2022-06-01 00:00:08.192,2022-06-01 00:00:08.192,31818.240234,31839.039062,31818.230469,31834.810547,41.377739,1654041719999,1.316950e+06,757,32.748501,1.042261e+06,BTCUSDT
2022-06-01 00:02:19.264,2022-06-01 00:02:19.264,31849.890625,31849.949219,31839.089844,31841.339844,24.777130,1654041839999,7.890021e+05,590,11.237340,3.578254e+05,BTCUSDT
2022-06-01 00:04:30.336,2022-06-01 00:04:30.336,31841.339844,31864.529297,31841.330078,31863.740234,17.768009,1654041959999,5.660112e+05,458,15.824350,5.040853e+05,BTCUSDT
2022-06-01 00:06:41.408,2022-06-01 00:06:41.408,31856.960938,31873.289062,31856.960938,31872.839844,9.692930,1654042079999,3.088801e+05,444,7.077800,2.255352e+05,BTCUSDT
2022-06-01 00:08:52.480,2022-06-01 00:08:52.480,31857.289062,31864.650391,31835.189453,31835.199219,21.587530,1654042199999,6.875410e+05,709,9.266330,2.951517e+05,BTCUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-16 22:50:00.000,2023-10-16 22:50:00.000,28525.019531,28530.919922,28512.259766,28512.269531,25.489479,1697496659999,7.269004e+05,608,5.580930,1.591692e+05,BTCUSDT
2023-10-16 22:51:00.000,2023-10-16 22:51:00.000,28512.269531,28533.939453,28511.509766,28525.019531,13.570990,1697496719999,3.870672e+05,569,8.962000,2.556054e+05,BTCUSDT
2023-10-16 22:52:00.000,2023-10-16 22:52:00.000,28525.009766,28525.019531,28508.830078,28508.830078,25.151211,1697496779999,7.171551e+05,585,4.109110,1.171667e+05,BTCUSDT


In [4]:
data = calc_utils.calc_RSI(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 332550 entries, 2022-06-01 00:00:08.192000 to 2023-10-16 22:54:00
Data columns (total 13 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   open_time                     332550 non-null  datetime64[ns]
 1   open                          332550 non-null  float32       
 2   high                          332550 non-null  float32       
 3   low                           332550 non-null  float32       
 4   close                         332550 non-null  float32       
 5   volume                        332550 non-null  float32       
 6   close_time                    332550 non-null  int64         
 7   quote_asset_volume            332550 non-null  float32       
 8   number_of_trades              332550 non-null  int16         
 9   taker_buy_base_asset_volume   332550 non-null  float32       
 10  taker_buy_quote_asset_volume  332550 no

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi
ix_open_time,,,,,,,,,,,,,
2022-06-01 00:00:08.192,2022-06-01 00:00:08.192,31818.240234,31839.039062,31818.230469,31834.810547,41.377739,1654041719999,1.316950e+06,757,32.748501,1.042261e+06,BTCUSDT,NaN
2022-06-01 00:02:19.264,2022-06-01 00:02:19.264,31849.890625,31849.949219,31839.089844,31841.339844,24.777130,1654041839999,7.890021e+05,590,11.237340,3.578254e+05,BTCUSDT,NaN
2022-06-01 00:04:30.336,2022-06-01 00:04:30.336,31841.339844,31864.529297,31841.330078,31863.740234,17.768009,1654041959999,5.660112e+05,458,15.824350,5.040853e+05,BTCUSDT,NaN
2022-06-01 00:06:41.408,2022-06-01 00:06:41.408,31856.960938,31873.289062,31856.960938,31872.839844,9.692930,1654042079999,3.088801e+05,444,7.077800,2.255352e+05,BTCUSDT,NaN
2022-06-01 00:08:52.480,2022-06-01 00:08:52.480,31857.289062,31864.650391,31835.189453,31835.199219,21.587530,1654042199999,6.875410e+05,709,9.266330,2.951517e+05,BTCUSDT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-16 22:50:00.000,2023-10-16 22:50:00.000,28525.019531,28530.919922,28512.259766,28512.269531,25.489479,1697496659999,7.269004e+05,608,5.580930,1.591692e+05,BTCUSDT,66.246613
2023-10-16 22:51:00.000,2023-10-16 22:51:00.000,28512.269531,28533.939453,28511.509766,28525.019531,13.570990,1697496719999,3.870672e+05,569,8.962000,2.556054e+05,BTCUSDT,69.108360
2023-10-16 22:52:00.000,2023-10-16 22:52:00.000,28525.009766,28525.019531,28508.830078,28508.830078,25.151211,1697496779999,7.171551e+05,585,4.109110,1.171667e+05,BTCUSDT,61.928581


In [5]:
data = calc_utils.calc_ema_periods(data, periods_of_time=[times_regression_PnL, 200])
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 332550 entries, 2022-06-01 00:00:08.192000 to 2023-10-16 22:54:00
Data columns (total 17 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   open_time                     332550 non-null  datetime64[ns]
 1   open                          332550 non-null  float32       
 2   high                          332550 non-null  float32       
 3   low                           332550 non-null  float32       
 4   close                         332550 non-null  float32       
 5   volume                        332550 non-null  float32       
 6   close_time                    332550 non-null  int64         
 7   quote_asset_volume            332550 non-null  float32       
 8   number_of_trades              332550 non-null  int16         
 9   taker_buy_base_asset_volume   332550 non-null  float32       
 10  taker_buy_quote_asset_volume  332550 no

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_120p,ema_120p_diff,ema_200p,ema_200p_diff
ix_open_time,,,,,,,,,,,,,,,,,
2022-06-01 00:00:08.192,2022-06-01 00:00:08.192,31818.240234,31839.039062,31818.230469,31834.810547,41.377739,1654041719999,1.316950e+06,757,32.748501,1.042261e+06,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2022-06-01 00:02:19.264,2022-06-01 00:02:19.264,31849.890625,31849.949219,31839.089844,31841.339844,24.777130,1654041839999,7.890021e+05,590,11.237340,3.578254e+05,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2022-06-01 00:04:30.336,2022-06-01 00:04:30.336,31841.339844,31864.529297,31841.330078,31863.740234,17.768009,1654041959999,5.660112e+05,458,15.824350,5.040853e+05,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2022-06-01 00:06:41.408,2022-06-01 00:06:41.408,31856.960938,31873.289062,31856.960938,31872.839844,9.692930,1654042079999,3.088801e+05,444,7.077800,2.255352e+05,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2022-06-01 00:08:52.480,2022-06-01 00:08:52.480,31857.289062,31864.650391,31835.189453,31835.199219,21.587530,1654042199999,6.875410e+05,709,9.266330,2.951517e+05,BTCUSDT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-16 22:50:00.000,2023-10-16 22:50:00.000,28525.019531,28530.919922,28512.259766,28512.269531,25.489479,1697496659999,7.269004e+05,608,5.580930,1.591692e+05,BTCUSDT,66.246613,28434.847933,0.272277,28423.755821,0.311408
2023-10-16 22:51:00.000,2023-10-16 22:51:00.000,28512.269531,28533.939453,28511.509766,28525.019531,13.570990,1697496719999,3.870672e+05,569,8.962000,2.556054e+05,BTCUSDT,69.108360,28436.338373,0.311859,28424.763421,0.352707
2023-10-16 22:52:00.000,2023-10-16 22:52:00.000,28525.009766,28525.019531,28508.830078,28508.830078,25.151211,1697496779999,7.171551e+05,585,4.109110,1.171667e+05,BTCUSDT,61.928581,28437.536583,0.250702,28425.599905,0.292800


In [6]:
data = utils.regression_PnL(
    data=data,
    label=myenv.label,
    diff_percent=float(stop_loss),
    max_regression_profit_and_loss=int(times_regression_PnL),
    drop_na=True,
    drop_calc_cols=True,
    strategy=None)
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 332231 entries, 2022-06-01 07:14:51.520000 to 2023-10-16 20:54:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   open_time                     332231 non-null  datetime64[ns]
 1   open                          332231 non-null  float32       
 2   high                          332231 non-null  float32       
 3   low                           332231 non-null  float32       
 4   close                         332231 non-null  float32       
 5   volume                        332231 non-null  float32       
 6   close_time                    332231 non-null  int64         
 7   quote_asset_volume            332231 non-null  float32       
 8   number_of_trades              332231 non-null  int16         
 9   taker_buy_base_asset_volume   332231 non-null  float32       
 10  taker_buy_quote_asset_volume  332231 no

In [7]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 332231 entries, 2022-06-01 07:14:51.520000 to 2023-10-16 20:54:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   open_time                     332231 non-null  datetime64[ns]
 1   open                          332231 non-null  float32       
 2   high                          332231 non-null  float32       
 3   low                           332231 non-null  float32       
 4   close                         332231 non-null  float32       
 5   volume                        332231 non-null  float32       
 6   close_time                    332231 non-null  int64         
 7   quote_asset_volume            332231 non-null  float32       
 8   number_of_trades              332231 non-null  int16         
 9   taker_buy_base_asset_volume   332231 non-null  float32       
 10  taker_buy_quote_asset_volume  332231 no

In [8]:
perc_data_label = data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(data)
perc_data_label

,open_time,perc
status,,
CAI_1.0,61253,0.184369
ESTAVEL,211276,0.635931
SOBE_1.0,59702,0.179700


In [9]:
train_data = data[(data['open_time'] >= start_train_date) & (data['open_time'] < start_test_date)]
train_data = train_data.sort_values(myenv.date_features)
train_data

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_120p,ema_120p_diff,ema_200p,ema_200p_diff,status
ix_open_time,,,,,,,,,,,,,,,,,,
2022-06-01 07:14:51.520,2022-06-01 07:14:51.520,31581.070312,31584.710938,31562.119141,31584.710938,34.212029,1654067759999,1.080028e+06,482,24.837780,784066.187500,BTCUSDT,50.023926,31618.636266,-0.107295,31645.749588,-0.192881,ESTAVEL
2022-06-01 07:17:02.592,2022-06-01 07:17:02.592,31543.820312,31543.820312,31528.669922,31529.990234,29.505920,1654067939999,9.304631e+05,424,6.523770,205700.625000,BTCUSDT,39.735523,31617.117771,-0.275571,31644.419546,-0.361610,ESTAVEL
2022-06-01 07:19:13.664,2022-06-01 07:19:13.664,31529.250000,31530.519531,31528.000000,31530.070312,9.161320,1654068059999,2.888466e+05,312,4.410390,139053.546875,BTCUSDT,39.755051,31615.627555,-0.270617,31643.107725,-0.357226,ESTAVEL
2022-06-01 07:21:24.736,2022-06-01 07:21:24.736,31523.310547,31523.310547,31515.000000,31519.990234,12.236990,1654068179999,3.856867e+05,412,5.610390,176823.031250,BTCUSDT,38.082287,31613.991251,-0.297340,31641.697461,-0.384642,ESTAVEL
2022-06-01 07:23:35.808,2022-06-01 07:23:35.808,31524.859375,31524.869141,31514.900391,31520.490234,16.176809,1654068299999,5.098693e+05,388,5.708530,179916.328125,BTCUSDT,38.221149,31612.392426,-0.290716,31640.311164,-0.378697,ESTAVEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 23:50:24.512,2022-12-31 23:50:24.512,16537.339844,16538.730469,16535.720703,16536.310547,37.044609,1672530719999,6.126175e+05,1661,19.214121,317753.156250,BTCUSDT,53.089970,16547.410585,-0.067080,16556.995784,-0.124934,ESTAVEL
2022-12-31 23:52:35.584,2022-12-31 23:52:35.584,16537.939453,16539.699219,16536.900391,16537.650391,25.318230,1672530839999,4.187246e+05,1313,12.416170,205347.484375,BTCUSDT,53.921921,16547.249259,-0.058009,16556.803293,-0.115680,ESTAVEL
2022-12-31 23:54:46.656,2022-12-31 23:54:46.656,16540.419922,16540.919922,16537.710938,16539.279297,36.236832,1672530959999,5.993276e+05,1551,16.202690,267981.312500,BTCUSDT,54.967575,16547.117524,-0.047369,16556.628925,-0.104790,ESTAVEL


In [10]:
test_data = data[data['open_time'] >= start_test_date]
test_data = test_data.sort_values(myenv.date_features)
test_data

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_120p,ema_120p_diff,ema_200p,ema_200p_diff,status
ix_open_time,,,,,,,,,,,,,,,,,,
2023-01-01 00:01:19.872,2023-01-01 00:01:19.872,16539.310547,16541.169922,16534.519531,16536.429688,62.901970,1672531379999,1.040248e+06,1930,20.739990,3.430094e+05,BTCUSDT,51.851116,16546.713800,-0.062152,16556.103172,-0.118829,ESTAVEL
2023-01-01 00:03:30.944,2023-01-01 00:03:30.944,16534.119141,16536.080078,16527.509766,16535.380859,144.453690,1672531499999,2.388081e+06,3795,76.524208,1.265080e+06,BTCUSDT,51.070633,16546.526478,-0.067359,16555.896980,-0.123920,ESTAVEL
2023-01-01 00:05:42.016,2023-01-01 00:05:42.016,16536.759766,16540.429688,16534.199219,16535.910156,96.455711,1672531619999,1.595134e+06,2348,42.826832,7.082578e+05,BTCUSDT,51.467655,16546.351002,-0.063101,16555.698106,-0.119524,ESTAVEL
2023-01-01 00:07:53.088,2023-01-01 00:07:53.088,16533.980469,16534.789062,16528.000000,16529.279297,95.135521,1672531739999,1.572652e+06,2619,48.513111,8.019371e+05,BTCUSDT,46.389351,16546.068825,-0.101471,16555.435233,-0.157990,ESTAVEL
2023-01-01 00:10:04.160,2023-01-01 00:10:04.160,16525.339844,16527.419922,16523.980469,16525.839844,43.337151,1672531919999,7.161810e+05,2011,24.285870,4.013480e+05,BTCUSDT,43.966053,16545.734462,-0.120240,16555.140751,-0.176990,ESTAVEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-16 20:50:00.000,2023-10-16 20:50:00.000,28459.890625,28459.900391,28440.000000,28440.009766,23.171310,1697489459999,6.591615e+05,675,8.309560,2.363387e+05,BTCUSDT,35.214493,28488.633197,-0.170677,28435.770539,0.014908,ESTAVEL
2023-10-16 20:51:00.000,2023-10-16 20:51:00.000,28440.000000,28440.009766,28432.060547,28436.900391,9.337960,1697489519999,2.655248e+05,468,3.899510,1.108789e+05,BTCUSDT,34.673096,28487.778109,-0.178595,28435.781781,0.003934,ESTAVEL
2023-10-16 20:52:00.000,2023-10-16 20:52:00.000,28436.900391,28436.910156,28430.849609,28436.429688,22.613220,1697489579999,6.429938e+05,555,5.086040,1.446115e+05,BTCUSDT,34.586403,28486.929375,-0.177273,28435.788228,0.002256,ESTAVEL


In [11]:
# BTCUSDT 1h best params: close,volume,quote_asset_volume,number_of_trades,rsi
# numeric_features = 'volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'.split(',')
#text_numeric_features = 'close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'
text_numeric_features = f'number_of_trades,rsi,ema_{int(times_regression_PnL)}p,ema_200p'
numeric_features = text_numeric_features.split(',')
print(f'Numeric Features: {numeric_features} - size: {len(numeric_features)}\n')

combination_numeric_features = []
if apply_combination_features:
	for size in range(1, len(numeric_features) + 1): 
		comb = map(list, combinations(numeric_features, size))	
		for c in comb:
			res = ''
			for j in c:
				res += f'{j},'
			combination_numeric_features.append(res[0:len(res) - 1])
else:
	combination_numeric_features = [text_numeric_features]

print(f'Combination Numeric Features: \n{combination_numeric_features}')

Numeric Features: ['number_of_trades', 'rsi', 'ema_120p', 'ema_200p'] - size: 4

Combination Numeric Features: 
['number_of_trades', 'rsi', 'ema_120p', 'ema_200p', 'number_of_trades,rsi', 'number_of_trades,ema_120p', 'number_of_trades,ema_200p', 'rsi,ema_120p', 'rsi,ema_200p', 'ema_120p,ema_200p', 'number_of_trades,rsi,ema_120p', 'number_of_trades,rsi,ema_200p', 'number_of_trades,ema_120p,ema_200p', 'rsi,ema_120p,ema_200p', 'number_of_trades,rsi,ema_120p,ema_200p']


In [13]:
from datetime import datetime
import os
# fix_imbalance_method: condensednearestneighbour, editednearestneighborus, repeatededitednearestneighbours, allknn, instancehardnessthreshold, nearmiss, neighbourhoodcleaningrule, onesidedselection, randomundersampler, tomeklinks, randomoversampler, smote, smotenc, smoten, adasyn, borderlinesmote, kmeanssmote, svmsmote, smoteenn, smotetomek.
# 'condensednearestneighbour,editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
# 'smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
# imbalance_list = 'editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
imbalance_list = 'instancehardnessthreshold,smoteenn,repeatededitednearestneighbours,allknn'.split(',')
simulation_results_filename = f'./resultado_simulacao_{symbol}_{interval}.csv'

df_resultado_simulacao = pd.DataFrame()
has_simulation_file = os.path.exists(simulation_results_filename)
if has_simulation_file:
  df_resultado_simulacao = pd.read_csv(simulation_results_filename, sep=';')

for aux_numeric_features in combination_numeric_features:
  experiement = ClassificationExperiment()
  features = []
  features += ['open_time', myenv.label]
  features += aux_numeric_features.split(',')
  print(f'features: {features}')
  for fix_imbalance_method in imbalance_list:
    if has_simulation_file:
      chave = (df_resultado_simulacao['symbol'] == symbol) & \
          (df_resultado_simulacao['interval'] == interval) & \
          (df_resultado_simulacao['estimator'] == estimator) & \
          (df_resultado_simulacao['fix_imbalance_method'] == fix_imbalance_method) & \
          (df_resultado_simulacao['features'] == str(features))

      if chave.sum() > 0:
        print(f'fix_imbalance_method: {fix_imbalance_method} already exists')
        continue

    try:
      print(f'fix_imbalance_method: {fix_imbalance_method}')
      setup = experiement.setup(
          data=train_data[features].copy(),
          train_size=myenv.train_size,
          target=myenv.label,
          numeric_features=aux_numeric_features.split(','),
          date_features=['open_time'],
          create_date_columns=["hour", "day", "month"],
          data_split_shuffle=False,
          data_split_stratify=False,
          fix_imbalance=True,
          fix_imbalance_method=fix_imbalance_method,
          remove_outliers=True,
          fold_strategy='timeseries', 
          fold=3,
          session_id=123,
          normalize=normalize,
          use_gpu=False,
          verbose=True,
          n_jobs=20,
          log_experiment=False,
      )

      if _compare_models:
        best = setup.compare_models()
        estimator = setup.pull().index[0]
        print('Estimator: ' + estimator)
      else:
        best = setup.create_model(estimator, n_jobs=20)

      if tune_model:
        best = setup.tune_model(best)

      # predict on test set
      # holdout_pred = setup.predict_model(best)
      # print('Holdout Score:', holdout_pred['prediction_score'].mean())
      # print('Holdout Score Group:\n', holdout_pred[[myenv.label, 'prediction_score']].groupby(myenv.label).mean())

      predict = setup.predict_model(best, data=test_data.drop(columns=[myenv.label]))
      predict[myenv.label] = test_data[myenv.label]
      predict['_score'] = predict['prediction_label'] == predict[myenv.label]
      # print('Predict Score Mean:', predict['_score'].mean())
      # print('Predict Score Mean Group:\n', predict[[myenv.label, '_score']].groupby(myenv.label).mean())

      final_model = setup.finalize_model(best)

      ajusted_test_data = test_data.drop(myenv.label, axis=1)
      df_final_predict, res_score = utils.validate_score_test_data(
          setup,
          final_model,
          myenv.label,
          test_data,
          ajusted_test_data)

      # df_final_predict.info()
      # print('Final Score Mean:', res_score.mean().values[0])
      # print('Final Score Group:\n', res_score)

      start_test_date = df_final_predict['open_time'].min()
      end_test_date = df_final_predict['open_time'].max()

      # print('Simule Trading:')
      # print(f'Min Data: {start_test_date}')
      # print(f'Max Data: {end_test_date}')
      saldo_inicial = 100.0
      saldo_final = utils.simule_trading_crypto2(df_final_predict, start_test_date, end_test_date, saldo_inicial, stop_loss)
      print(f'>>>> Saldo Final: {saldo_final} - features: {features}\n\n')

      result_simulado = {}
      result_simulado['date'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
      result_simulado['symbol'] = symbol
      result_simulado['interval'] = interval
      result_simulado['estimator'] = estimator
      result_simulado['fix_imbalance_method'] = fix_imbalance_method
      result_simulado['stop_loss'] = stop_loss
      result_simulado['times_regression_profit_and_loss'] = times_regression_PnL
      result_simulado['features'] = features
      result_simulado['final_value'] = saldo_final

      if res_score is not None:
        result_simulado['score'] = ''
        for i in range(0, len(res_score.index.values)):
          result_simulado['score'] += f'[{res_score.index.values[i]}={res_score["_score"].values[i]:.2f}]'

      df_resultado_simulacao = pd.concat([df_resultado_simulacao, pd.DataFrame([result_simulado])], ignore_index=True)
      df_resultado_simulacao.sort_values('final_value', inplace=True)

      df_resultado_simulacao.to_csv(simulation_results_filename, sep=';', index=False)
    except Exception as e:
      print(e)
      continue

features: ['open_time', 'status', 'number_of_trades']
fix_imbalance_method: instancehardnessthreshold


,Description,Value
0,Session id,123
1,Target,status
2,Target type,Multiclass
3,Target mapping,"CAI_1.0: 0, ESTAVEL: 1, SOBE_1.0: 2"
4,Original data shape,"(140866, 3)"
5,Transformed data shape,"(119294, 5)"
6,Transformed train set shape,"(77034, 5)"
7,Transformed test set shape,"(42260, 5)"
8,Numeric features,1
9,Date features,1


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.3246,0.5217,0.3246,0.3293,0.2483,-0.0087,-0.0114
1,0.3184,0.4794,0.3184,0.3423,0.3265,-0.0406,-0.0412
2,0.4059,0.5636,0.4059,0.5106,0.4371,0.0684,0.0728
Mean,0.3496,0.5216,0.3496,0.3940,0.3373,0.0064,0.0068
Std,0.0399,0.0344,0.0399,0.0826,0.0775,0.0458,0.0483


>>>> Saldo Final: 678.8794262516603 - features: ['open_time', 'status', 'number_of_trades']


fix_imbalance_method: smoteenn
'DataFrame' object has no attribute 'append'
fix_imbalance_method: repeatededitednearestneighbours


,Description,Value
0,Session id,123
1,Target,status
2,Target type,Multiclass
3,Target mapping,"CAI_1.0: 0, ESTAVEL: 1, SOBE_1.0: 2"
4,Original data shape,"(140866, 3)"
5,Transformed data shape,"(77332, 5)"
6,Transformed train set shape,"(35072, 5)"
7,Transformed test set shape,"(42260, 5)"
8,Numeric features,1
9,Date features,1


...

>>>> Saldo Final: 460.2281144160991 - features: ['open_time', 'status', 'number_of_trades', 'ema_120p']


fix_imbalance_method: smoteenn
'DataFrame' object has no attribute 'append'
fix_imbalance_method: repeatededitednearestneighbours


,Description,Value
0,Session id,123
1,Target,status
2,Target type,Multiclass
3,Target mapping,"CAI_1.0: 0, ESTAVEL: 1, SOBE_1.0: 2"
4,Original data shape,"(140866, 4)"
5,Transformed data shape,"(85002, 6)"
6,Transformed train set shape,"(42742, 6)"
7,Transformed test set shape,"(42260, 6)"
8,Numeric features,2
9,Date features,1


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:24:56
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,K Neighbors Classifier


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 